### Notebook to evaluate results

In [1]:
import os
import cv2
import matplotlib.pyplot as plt
from imutils import paths
import time
import glob

import tifffile as tiff
import numpy as np
from PIL import Image

import numpy as np
from sklearn.metrics import jaccard_score

In [2]:
# CONFIGURE PATHS
GD_PATH = os.getcwd() + "/"
PLOT_PATH = GD_PATH + "plots/"

# TEST
# Task 1: 
TEST_IMG_DIR = "/home/shymon/datasets/mapai_full/task1_test/images/"
TEST_MASK_DIR = "/home/shymon/datasets/mapai_full/task1_test/masks/"

# TEST
test_images = sorted(list(paths.list_images(TEST_IMG_DIR)))
test_masks = sorted(list(paths.list_images(TEST_MASK_DIR)))

PREDICTIONS_DIR = GD_PATH + "predictions/"
REGULARIZATION_DIR = GD_PATH + "regularizations/"

# read predictions
predictions = glob.glob(PREDICTIONS_DIR + "*.tif")
predictions.sort()

# read regularizations
regularizations = glob.glob(REGULARIZATION_DIR + "*.tif")
regularizations.sort()

print("# TEST IMAGES: ", len(test_images))
print("# PREDICTIONS: ", len(predictions))
print("# REGULARIZATIONS: ", len(predictions))

# Project Regularization directory
projectRegDir = GD_PATH + "projectRegularization" + "/"

ptw = projectRegDir + "pretrained_weights" + "/"

# GET THE PATHS FOR TRAINED GAN MODELS
ENCODER = ptw + "E140000_e1"
GENERATOR = ptw + "E140000_net"

# print(ENCODER)
# print(GENERATOR)

# TEST IMAGES:  1368
# PREDICTIONS:  1368
# REGULARIZATIONS:  1368


### Calculate Intersection over Union on the test set

In [3]:
def iou(gt_mask, pred_mask):

    """
    Calculates the intersection over union (BIoU) between two binary semantic segmentation masks.
    
    Arguments:
    mask1 -- a 2D numpy array representing the first mask
    mask2 -- a 2D numpy array representing the second mask
    
    Returns:
    iou -- a float representing the BIoU between the two masks
    """

    intersection  = np.logical_and(gt_mask, pred_mask).sum()
    union = np.logical_or(gt_mask, pred_mask).sum()
    iou_score = intersection / union if union != 0 else np.nan

    return iou_score

def biou(segA, segB, boundary_width=1):
    """
    Calculate the Boundary Intersection over Union (BIoU) metric between two binary segmentation masks.

    Parameters:
    segA (numpy array): A 2-dimensional binary numpy array representing the first segmentation mask.
    segB (numpy array): A 2-dimensional binary numpy array representing the second segmentation mask.
    boundary_width (int): The width of the boundary region to be included in the calculation (default is 1).

    Returns:
    float: The BIoU metric between the two segmentation masks.
    """

    # Compute the boundaries of the segmentation masks
    boundaryA = np.zeros_like(segA)
    boundaryA[:,boundary_width:-boundary_width] = segA[:,boundary_width:-boundary_width] ^ segA[:, :-2*boundary_width] ^ segA[:, 2*boundary_width:]
    boundaryA[boundary_width:-boundary_width,:] = boundaryA[boundary_width:-boundary_width,:] ^ segA[:-2*boundary_width,:] ^ segA[2*boundary_width:,:]

    boundaryB = np.zeros_like(segB)
    boundaryB[:,boundary_width:-boundary_width] = segB[:,boundary_width:-boundary_width] ^ segB[:, :-2*boundary_width] ^ segB[:, 2*boundary_width:]
    boundaryB[boundary_width:-boundary_width,:] = boundaryB[boundary_width:-boundary_width,:] ^ segB[:-2*boundary_width,:] ^ segB[2*boundary_width:,:]

    # Compute the coordinates of the intersection boundary
    intersection_boundary = boundaryA & boundaryB

    # Compute the coordinates of the union boundary
    union_boundary = boundaryA | boundaryB

    # Compute the area of intersection boundary
    intersection_boundary_area = np.count_nonzero(intersection_boundary)

    # Compute the area of union boundary
    union_boundary_area = np.count_nonzero(union_boundary)

    # Compute the intersection and union of the interior regions
    intersection = np.logical_and(segA, segB)
    union = np.logical_or(segA, segB)

    # Compute the area of intersection and union of the interior regions
    intersection_area = np.count_nonzero(intersection)
    union_area = np.count_nonzero(union)

    # Compute the BIoU metric
    biou = (intersection_area + intersection_boundary_area) / (union_area + union_boundary_area + 1e-6)

    return biou

# To read the original test images from MapAI
def test2arr(tif_img):
    img = tiff.imread(tif_img)
    arr = np.array(img)
    return arr

# To read the predictions and regularizations
def pr2arr(tif_img):
    img = tiff.imread(tif_img)
    img = img / 255
    img = cv2.resize(img, (500, 500))
    arr = np.array(img)
    arr = arr.astype(np.uint8)
    return arr


### Evaluation on single image by choice

(1) Without regularization

In [4]:
n = 900

jaccard_sklearn = jaccard_score(test2arr(test_masks[n]), pr2arr(predictions[n]), average='micro')
print("Jaccard score or IoU with Scikit-learn: ", round(jaccard_sklearn, 4))

iou_man = iou(test2arr(test_masks[n]), pr2arr(predictions[n]))
print("Jaccard score or IoU with manual function: ", round(iou_man, 4))

biou_man = biou(test2arr(test_masks[n]), pr2arr(predictions[n]))
print("Boundary Intersection over Union: ", round(biou_man, 4))

Jaccard score or IoU with Scikit-learn:  0.7001
Jaccard score or IoU with manual function:  0.7001
Boundary Intersection over Union:  0.6959


(2) With regularization

In [5]:
n = 900

jaccard_sklearn = jaccard_score(test2arr(test_masks[n]), pr2arr(regularizations[n]), average='micro')
print("Jaccard score or IoU with Scikit-learn: ", round(jaccard_sklearn, 4))

iou_man = iou(test2arr(test_masks[n]), pr2arr(regularizations[n]))
print("Jaccard score or IoU with manual function: ", round(iou_man, 4))

biou_man = biou(test2arr(test_masks[n]), pr2arr(regularizations[n]))
print("Boundary Intersection over Union: ", round(biou_man, 4))

Jaccard score or IoU with Scikit-learn:  0.6841
Jaccard score or IoU with manual function:  0.6841
Boundary Intersection over Union:  0.6801


### Evaluation on entire MapAI dataset

(1) Without regularization

In [6]:
iou_mapai = np.array([])
biou_mapai = np.array([])

for n in range(len(test_masks)):
    
    # Calculate metrics

    # IoU
    iou_single = iou(test2arr(test_masks[n]), pr2arr(predictions[n]))

    # BIoU
    biou_single = biou(test2arr(test_masks[n]), pr2arr(predictions[n]))

    # Append to whole array
    iou_mapai = np.append(iou_mapai, iou_single)
    biou_mapai = np.append(biou_mapai, biou_single)

#iou_mapai = iou_mapai[iou_mapai != 0]
#biou_mapai = biou_mapai[biou_mapai != 0]

print("Evaluation without regularization: ")
print("Mean IoU for Task 1: ", round(np.nanmean(iou_mapai), 4))
print("Mean BIoU for Task 1: ", round(np.nanmean(biou_mapai), 4))
print("S metric for Task 1: ", round(((np.nanmean(biou_mapai) + np.nanmean(iou_mapai)) / 2 ), 4))

Evaluation without regularization: 
Mean IoU for Task 1:  0.3995
Mean BIoU for Task 1:  0.3766
S metric for Task 1:  0.3881


(2) With regularization

In [7]:
iou_mapai = np.array([])
biou_mapai = np.array([])

for n in range(len(test_masks)):
    
    # Calculate metrics

    # IoU
    iou_single = iou(test2arr(test_masks[n]), pr2arr(regularizations[n]))

    # BIoU
    biou_single = biou(test2arr(test_masks[n]), pr2arr(regularizations[n]))

    # Append to whole array
    iou_mapai = np.append(iou_mapai, iou_single)
    biou_mapai = np.append(biou_mapai, biou_single)

#iou_mapai = iou_mapai[iou_mapai != 0]
#biou_mapai = biou_mapai[biou_mapai != 0]

print("Mean IoU for Task 1: ", round(np.nanmean(iou_mapai), 4))
print("Mean BIoU for Task 1: ", round(np.nanmean(biou_mapai), 4))
print("S metric for Task 1: ", round(((np.nanmean(biou_mapai) + np.nanmean(iou_mapai)) / 2 ), 4))

Mean IoU for Task 1:  0.4017
Mean BIoU for Task 1:  0.378
S metric for Task 1:  0.3898
